In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Import libraries

In [2]:
# Import the libraries
import math
import copy
import itertools
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

# Loading a Dataset

In [3]:
# Import the dataset
train_calc = pd.read_csv('calc_case_description_train_set.csv')
test_calc = pd.read_csv('calc_case_description_test_set.csv')
train_mass = pd.read_csv('mass_case_description_train_set.csv')
test_mass = pd.read_csv('mass_case_description_test_set.csv')

calc = train_calc.values.tolist() + test_calc.values.tolist()
calc = pd.DataFrame(calc, columns = train_calc.columns)
mass = train_mass.values.tolist() + test_mass.values.tolist()
mass = pd.DataFrame(mass, columns = train_mass.columns)

# Data Processing Settings

In [4]:
# embedding sizes
calc_type_embedding_size = 0
calc_dist_embedding_size = 0
mass_shape_embedding_size = 0
mass_margins_embedding_size = 0

In [5]:
# Function to create dictionaries for categorical features
def create_embedding_dict(series):
    unique_values = series.unique()
    embedding_dict = {value: i for i, value in enumerate(unique_values)}
    return embedding_dict

In [6]:
def process(data):
        # make a copy of the data to avoid SettingWithCopyWarning
        data = data.copy()
        
        # set the limitations on the numerical columns
        try:
                data['breast density'] = data['breast density'].clip(1, 4)
        except KeyError:
                data['breast_density'] = data['breast_density'].clip(1, 4)
        data['abnormality id'] = data['abnormality id'].clip(0)
        data['assessment'] = data['assessment'].clip(0, 5)
        data['subtlety'] = data['subtlety'].clip(1, 5)
        
        # change the name of index
        data.index = data['patient_id'] + '_' + data['image view'] + '_' \
        + data['left or right breast'] + '_' + data['abnormality id'].astype(str)

        # Remove useless columns
        data = data[data.columns.drop(list(data.filter(regex='file path')) 
                + ['image view', 'patient_id', 'left or right breast', 'abnormality type'])]

        # Fill NaN values with appropriate placeholders
        try:
                data['calc type'] = data['calc type'].fillna('None')
                data['calc distribution'] = data['calc distribution'].fillna('None')
        except KeyError:
                data['mass shape'] = data['mass shape'].fillna('None')
                data['mass margins'] = data['mass margins'].fillna('None')
        # '''
        # pathology :
        # BENIGN_WITHOUT_CALLBACK = 0
        # BENIGN = 1
        # MALIGNANT = 2
        # '''
        data['pathology'] = data['pathology'].map({'BENIGN_WITHOUT_CALLBACK': 0, 'BENIGN': 1, 'MALIGNANT': 2})
        
        # Create embedding dictionaries for categorical features
        # and define embedding sizes
        
        global calc_type_embedding_size
        global calc_dist_embedding_size
        global mass_shape_embedding_size
        global mass_margins_embedding_size
        
        try:
                calc_type_embedding_dict = create_embedding_dict(data['calc type'])
                calc_dist_embedding_dict = create_embedding_dict(data['calc distribution'])
                calc_type_embedding_size = len(calc_type_embedding_dict)
                calc_dist_embedding_size = len(calc_dist_embedding_dict)
        except KeyError:
                mass_shape_embedding_dict = create_embedding_dict(data['mass shape'])
                mass_margins_embedding_dict = create_embedding_dict(data['mass margins'])
                mass_shape_embedding_size = len(mass_shape_embedding_dict)
                mass_margins_embedding_size = len(mass_margins_embedding_dict)
        
        # Replace categorical values with their embedding indices        
        try:
                data['calc type'] = data['calc type'].map(calc_type_embedding_dict)
                data['calc distribution'] = data['calc distribution'].map(calc_dist_embedding_dict)
        except KeyError:
                data['mass shape'] = data['mass shape'].map(mass_shape_embedding_dict)
                data['mass margins'] = data['mass margins'].map(mass_margins_embedding_dict)
        
        # rename columns
        data.rename(columns={'abnormality id': 'number of abnormalities', 
                             'assessment' : 'overall BI-RADS assessment'}, inplace=True)
        try:
                data.rename(columns={'breast_density' : 'breast density'}, inplace=True)
                # split
        except KeyError:
                return data
        
        return data

# Dataprocessing

In [7]:
# Reload the csv files and dataset
train_calc = pd.read_csv('calc_case_description_train_set.csv')
test_calc = pd.read_csv('calc_case_description_test_set.csv')
train_mass = pd.read_csv('mass_case_description_train_set.csv')
test_mass = pd.read_csv('mass_case_description_test_set.csv')

calc = train_calc.values.tolist() + test_calc.values.tolist()
calc = pd.DataFrame(calc, columns = train_calc.columns)
mass = train_mass.values.tolist() + test_mass.values.tolist()
mass = pd.DataFrame(mass, columns = train_mass.columns)

In [8]:
# Execute the data process function
calc = process(calc)
mass = process(mass)
train_calc, test_calc = calc[:train_calc.shape[0]], calc[train_calc.shape[0]:]
train_mass, test_mass = mass[:train_mass.shape[0]], mass[train_mass.shape[0]:]

# Creating a Custom Dataset

In [9]:
# label maps
labels_map_assessment = {
    0: "0",
    1: "1",
    2: "2",
    3: "3",
    4: "4",
    5: "5"
}

labels_map_pathology = {
    0: "BENIGN_WITHOUT_CALLBACK",
    1: "BENIGN",
    2: "MALIGNANT"
}

In [10]:
class CreateDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Preparing data for training with DataLoaders

In [11]:
def get_dataloaders(train, test, type, batch_size):
    if type == "a":
        # calc, assessement prediction
        X_train = train.drop(['overall BI-RADS assessment', 'pathology'], axis=1).values
        y_train = train['overall BI-RADS assessment'].values
        X_test = test.drop(['overall BI-RADS assessment', 'pathology'], axis=1).values
        y_test = test['overall BI-RADS assessment'].values
    elif type == "p":
        # calc, pathology prediction
        X_train = train.drop('pathology', axis=1).values
        y_train = train['pathology'].values
        X_test = test.drop('pathology', axis=1).values
        y_test = test['pathology'].values
        
    # Assuming you have already preprocessed the data and split it into training and testing sets    
    train_dataset = CreateDataset(X_train, y_train)
    test_dataset = CreateDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_dataset, test_dataset, train_loader, test_loader

# Using cuda

In [12]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Deine the FNN Class

In [13]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, hidden_layers=1, activation="relu", weight_init="lecun"):
        super().__init__()
        self.linear_non_linear_stack = nn.Sequential().to(device)
        
        modules = []
        modules.append(nn.Linear(input_size, hidden_size))
        
        # weight initialization
        if weight_init == "lecun":
            pass
        elif weight_init == "zero":
            nn.init.zeros_(modules[0].weight)
        elif weight_init == "normal":
            nn.init.normal_(modules[0].weight, mean=0, std=0.01)
        elif weight_init == "xavier" or weight_init == "glorot":
            nn.init.xavier_normal_(modules[0].weight)
        elif weight_init == "kaiming" or weight_init == "he":
            nn.init.kaiming_normal_(modules[0].weight)
        
        # activation function
        if activation == "relu":
            modules.append(nn.ReLU())
        elif activation == "sigmoid":
            modules.append(nn.Sigmoid())
        elif activation == "tanh":
            modules.append(nn.Tanh())
        elif activation == "leaky_relu":
            modules.append(nn.LeakyReLU())
        
        for i in range(hidden_layers):
            ln = nn.Linear(hidden_size, hidden_size)
            
            if weight_init == "lecun":
                pass
            elif weight_init == "zero":
                nn.init.zeros_(ln.weight)
            elif weight_init == "normal":
                nn.init.normal_(ln.weight, mean=0, std=0.01)
            elif weight_init == "xavier" or weight_init == "glorot":
                nn.init.xavier_normal_(ln.weight)
            elif weight_init == "kaiming" or weight_init == "he":
                nn.init.kaiming_normal_(ln.weight)
            
            modules.append(ln)
            if activation == "relu":
                modules.append(nn.ReLU())
            elif activation == "sigmoid":
                modules.append(nn.Sigmoid())
            elif activation == "tanh":
                modules.append(nn.Tanh())
            elif activation == "leaky_relu":
                modules.append(nn.LeakyReLU())
                
        modules.append(nn.Linear(hidden_size, num_classes))
        
        self.linear_non_linear_stack = nn.Sequential(*modules).to(device)

    def forward(self, x):
        x = x.to(device)
        logits = self.linear_non_linear_stack(x)
        return logits

# Train and test the model

## Function for Train and Test

In [14]:
# train and test the model 
def train_loop(dataloader, model, loss_fn, optimizer, p="False", scheduler=None):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device).long()  # Move input and target tensors to the same device as the model
        # Compute prediction and loss
        pred = model(X).to(device)
        # print(pred.shape)
        # print(y.shape)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0 or batch == len(dataloader) - 1:
            loss, current = loss.item(), (batch + 1) * len(X)
            if batch == len(dataloader) - 1:
                current = size
            if p == True:
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, p="False"):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device).long()  # Move input and target tensors to the same device as the model
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    if p == True:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return round(100*correct, 1), test_loss

## Class for saving each models' information effectively

In [15]:
class Model:
    def __init__(self):
        pass

## Initial Settings

In [16]:
batch_size = 64
hidden_size = 512
learning_rate = 1e-3
epochs = 10

In [46]:
def l1_regularizer(model, l1_lambda):
    l1_loss = 0
    for param in model.parameters():
        l1_loss += torch.sum(torch.abs(param))
    return l1_lambda * l1_loss

In [47]:
def train_L1(dataloader, model, loss_fn, optimizer, l1_lambda, p="False", scheduler=None):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device).long()  # Move input and target tensors to the same device as the model
        # Compute prediction and loss
        pred = model(X).to(device)
        
        loss = loss_fn(pred, y)

        # Compute L1 regularization penalty
        l1_penalty = l1_regularizer(model, l1_lambda)

        # Combine original loss and L1 penalty
        loss = loss + l1_penalty
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0 or batch == len(dataloader) - 1:
            loss, current = loss.item(), (batch + 1) * len(X)
            if batch == len(dataloader) - 1:
                current = size
            if p == True:
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## features -> assessment (calc)

In [256]:
# default settings
fac = Model()

fac.epochs = epochs
fac.batch_size = batch_size
fac.train_dataset, fac.test_dataset, fac.train_dataloader, fac.test_dataloader = get_dataloaders(train_calc, test_calc, "a", batch_size)

fac.input_size = fac.train_dataloader.dataset.X.shape[1]
fac.hidden_size = hidden_size
fac.num_classes = 6

fac.lr = 1e-3

In [281]:
# default model
fac.model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
fac.loss_fn = nn.CrossEntropyLoss()
fac.optimizer = torch.optim.SGD(fac.model.parameters(), lr=fac.lr)

for t in range(fac.epochs):
    train_loop(fac.train_dataloader, fac.model, fac.loss_fn, fac.optimizer)
    accuracy = test_loop(fac.test_dataloader, fac.model, fac.loss_fn)[0]
fac.accuracy = accuracy
fac.accuracy

73.0

In [283]:
# Mini-batch gradient descent
for i in range(20):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fac.lr)

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [282]:
# SGD momentum
for i in range(20):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fac.lr, momentum=0.9)

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

New best model found with accuracy:  73.6
New best model found with accuracy:  74.5


In [285]:
# SGD Nestrov
for i in range(20):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fac.lr, momentum=0.9, nesterov=True)

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [292]:
# Adam
for i in range(20):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

New best model found with accuracy:  76.7


In [297]:
# Adagrad
for i in range(20):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adagrad(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [298]:
# Adadelta
for i in range(20):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adadelta(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [299]:
# Adamax
for i in range(20):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adamax(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [300]:
# RMSprop
for i in range(20):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

Adam

In [310]:
# Number of Neurons: int((input_size + num_classes)/2)
n = int((fac.input_size + fac.num_classes)/2)
for i in range(40):
    model = FNN(fac.input_size, n, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.hidden_size = n
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [311]:
# Number of Neurons: int(2/3*input_size + num_classes)
n = int(2/3*fac.input_size + fac.num_classes)
for i in range(40):
    model = FNN(fac.input_size, n, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.hidden_size = n
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

512

In [314]:
# Number of Layers: 2
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes, hidden_layers=2).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.hidden_layers = 2
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [315]:
# Number of Layers: 3
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes, hidden_layers=3).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.hidden_layers = 3
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

1

In [316]:
# Activation Function: leaky_relu (default is ReLU)
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes, activation="leaky_relu").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [317]:
# Activation Function: tanh (default is ReLU)
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes, activation="tanh").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [318]:
# Activation Function: sigmoid (default is ReLU)
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes, activation="sigmoid").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

ReLu

In [319]:
# weight_init : zero (default is lecun)
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes, weight_init="zero").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [320]:
# weight_init : normal (default is lecun)
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes, weight_init="normal").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [321]:
# weight_init : xavier (default is lecun)
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes, weight_init="xavier").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [322]:
# weight_init : kaiming (default is lecun)
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes, weight_init="kaiming").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

Lecun Normal

In [324]:
# Batch size: 32 (default is 64)
train_dataset, test_dataset, train_dataloader, test_dataloader = get_dataloaders(train_calc, test_calc, "a", 32)
 
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.batch_size = 32
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

New best model found with accuracy:  77.3


In [325]:
# Batch size: 128 (default is 64)
train_dataset, test_dataset, train_dataloader, test_dataloader = get_dataloaders(train_calc, test_calc, "a", 128)
 
for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.batch_size = 16
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

In [333]:
fac.train_dataset, fac.test_dataset, fac.train_dataloader, fac.test_dataloader = get_dataloaders(train_calc, test_calc, "a", 32)

32

In [338]:
# number of epochs (default): 10 -> increase until 3-
e = fac.epochs
for i in range(3):
    for j in range(10):
        model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        for t in range(e):
            train_loop(fac.train_dataloader, model, loss_fn, optimizer)
            accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
        
        if accuracy > fac.accuracy:
            print("New best epochs: ", e)
            print("New best model found with accuracy: ", accuracy)
            fac.accuracy = accuracy
            fac.epochs = e
            fac.model = model
            fac.loss_fn = loss_fn
            fac.optimizer = optimizer
    e += 1

In [343]:
# learning rate scheduler: StepLR
# learning rate: 0.001 (default) same as Adam's default

for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=fac.lr)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
    
    for t in range(fac.epochs):
        scheduler.step()
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

c:\Users\sonog\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [344]:
# learning rate scheduler: ReduceLROnPlateau
# learning rate: 0.001 (default) same as Adam's default

for i in range(40):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=fac.lr)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0)
    
    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer
    scheduler.step(accuracy)

No scheduler required

In [348]:
# L2 Regularization
# Define the hyperparameter search space
epochs_candidates = [10, 20, 30]
l2_lambda_candidates = [0.001, 0.01, 0.1]

for num_epochs, l2_lambda in itertools.product(epochs_candidates, l2_lambda_candidates):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

    for t in range(num_epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        print("New best hyperparameters: ", num_epochs, l2_lambda)
        
        fac.accuracy = accuracy
        
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer
        
        fac.epochs = num_epochs
        fac.l2_lambda = l2_lambda

In [487]:
# L1 Regularization
epochs_candidates = [10, 20, 30]
l1_lambda_candidates = [0.001, 0.01, 0.1]

for num_epochs, l1_lambda in itertools.product(epochs_candidates, l1_lambda_candidates):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l1_lambda)

    for t in range(num_epochs):
        train_L1(fac.train_dataloader, model, loss_fn, optimizer, l1_lambda)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        print("New best hyperparameters: ", num_epochs, l1_lambda)
        
        fac.accuracy = accuracy
        
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer
        
        fac.epochs = num_epochs
        fac.l1_lambda = l1_lambda

No regularization

Try with lower batch size then because the change in batch size improved the model

In [429]:
# Batch size: 16 (default is 64)
train_dataset, test_dataset, train_dataloader, test_dataloader = get_dataloaders(train_calc, test_calc, "a", 16)
 
for i in range(60):
    model = FNN(fac.input_size, fac.hidden_size, fac.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fac.epochs):
        train_loop(fac.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fac.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fac.accuracy = accuracy
        fac.train_dataset, fac.test_dataset, fac.train_dataloader, fac.test_dataloader = train_dataset, test_dataset, train_dataloader, test_dataloader
        fac.batch_size = 16
        fac.model = model
        fac.loss_fn = loss_fn
        fac.optimizer = optimizer

## features -> assessment (mass)

In [354]:
# default settings
fam = Model()

fam.epochs = epochs
fam.batch_size = batch_size
fam.train_dataset, fam.test_dataset, fam.train_dataloader, fam.test_dataloader = get_dataloaders(train_mass, test_mass, "a", batch_size)

fam.input_size = fam.train_dataloader.dataset.X.shape[1]
fam.hidden_size = hidden_size
fam.num_classes = 6

fam.lr = 1e-3

In [381]:
# default model
fam.model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
fam.loss_fn = nn.CrossEntropyLoss()
fam.optimizer = torch.optim.SGD(fam.model.parameters(), lr=fam.lr)

for t in range(fac.epochs):
    train_loop(fam.train_dataloader, fam.model, fam.loss_fn, fam.optimizer)
    accuracy = test_loop(fam.test_dataloader, fam.model, fam.loss_fn)[0]
fam.accuracy = accuracy
fam.accuracy

52.6

In [383]:
# Mini-batch gradient descent
for i in range(30):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fam.lr, momentum=0.9)

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fac.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

New best model found with accuracy:  53.7
New best model found with accuracy:  60.1


In [384]:
# SGD momentum
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fam.lr, momentum=0.9)

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [386]:
# SGD Nestrov
for i in range(20):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fam.lr, momentum=0.9, nesterov=True)

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [387]:
# Adam
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

New best model found with accuracy:  61.1
New best model found with accuracy:  61.9
New best model found with accuracy:  62.7
New best model found with accuracy:  63.0


In [388]:
# Adagrad
for i in range(20):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adagrad(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [389]:
# Adadelta
for i in range(20):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adadelta(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [390]:
# Adamax
for i in range(20):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adamax(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [391]:
# RMSprop
for i in range(20):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

New best model found with accuracy:  63.5


RMSprop

In [393]:
# learning rate scheduler: StepLR

for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
    
    for t in range(fam.epochs):
        scheduler.step()
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

c:\Users\sonog\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [394]:
# learning rate scheduler: ReduceLROnPlateau

for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0)
    
    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer
    scheduler.step(accuracy)

No scheduler required

In [396]:
# Number of Neurons: int((input_size + num_classes)/2)
n = int((fam.input_size + fam.num_classes)/2)
for i in range(40):
    model = FNN(fam.input_size, n, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.hidden_size = n
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [398]:
# Number of Neurons: int(2/3*input_size + num_classes)
n = int(2/3*fam.input_size + fam.num_classes)
for i in range(40):
    model = FNN(fam.input_size, n, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.hidden_size = n
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

Get the default number of neurons

In [401]:
# Batch size: 32 (default is 64)
train_dataset, test_dataset, train_dataloader, test_dataloader = get_dataloaders(train_mass, test_mass, "a", 32)
 
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.train_dataset, fam.test_dataset, fam.train_dataloader, fam.test_dataloader = train_dataset, test_dataset, train_dataloader, test_dataloader
        fam.batch_size = 32
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [402]:
# Batch size: 16 (default is 64)
train_dataset, test_dataset, train_dataloader, test_dataloader = get_dataloaders(train_mass, test_mass, "a", 16)
 
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.train_dataset, fam.test_dataset, fam.train_dataloader, fam.test_dataloader = train_dataset, test_dataset, train_dataloader, test_dataloader
        fam.batch_size = 16
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

No need to change the batch size. 

In [407]:
# Hidden Layers: 2
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes, hidden_layers=2).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.hidden_layers = 2
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

Hidden Layers = 1

In [408]:
# number of epochs (default): 10 -> increase until 3-
e = fam.epochs
for i in range(3):
    for j in range(10):
        model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.RMSprop(model.parameters())

        for t in range(e):
            train_loop(fam.train_dataloader, model, loss_fn, optimizer)
            accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
        
        if accuracy > fam.accuracy:
            print("New best epochs: ", e)
            print("New best model found with accuracy: ", accuracy)
            fam.accuracy = accuracy
            fam.epochs = e
            fam.model = model
            fam.loss_fn = loss_fn
            fam.optimizer = optimizer
    e += 1

10

In [417]:
# Activation Function: leaky_relu (default is ReLU)
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes, activation="leaky_relu").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [418]:
# Activation Function: tanh (default is ReLU)
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes, activation="tanh").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [420]:
# Activation Function: sigmoid (default is ReLU)
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes, activation="sigmoid").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

ReLu

In [421]:
# weight_init : zero (default is lecun)
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes, weight_init="zero").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [422]:
# weight_init : normal (default is lecun)
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes, weight_init="normal").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [423]:
# weight_init : xavier (default is lecun)
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes, weight_init="xavier").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

In [424]:
# weight_init : kaiming (default is lecun)
for i in range(40):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes, weight_init="kaiming").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

Lecun

In [425]:
# L2 Regularization
# Define the hyperparameter search space
epochs_candidates = [10, 20, 30]
l2_lambda_candidates = [0.001, 0.01, 0.1]

for num_epochs, l2_lambda in itertools.product(epochs_candidates, l2_lambda_candidates):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

    for t in range(num_epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        print("New best hyperparameters: ", num_epochs, l2_lambda)
        
        fam.accuracy = accuracy
        
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer
        
        fam.epochs = num_epochs
        fam.l2_lambda = l2_lambda

In [486]:
# L1 Regularization
epochs_candidates = [10, 20, 30]
l1_lambda_candidates = [0.001, 0.01, 0.1]

for num_epochs, l1_lambda in itertools.product(epochs_candidates, l1_lambda_candidates):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l1_lambda)

    for t in range(num_epochs):
        train_L1(fam.train_dataloader, model, loss_fn, optimizer, l1_lambda)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        print("New best hyperparameters: ", num_epochs, l1_lambda)
        
        fam.accuracy = accuracy
        
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer
        
        fam.epochs = num_epochs
        fam.l1_lambda = l1_lambda

No regularization so this is the best:

In [428]:
# RMSprop
for i in range(100):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

New best model found with accuracy:  64.0


In [430]:
# RMSprop
for i in range(100):
    model = FNN(fam.input_size, fam.hidden_size, fam.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fam.epochs):
        train_loop(fam.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fam.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fam.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fam.accuracy = accuracy
        fam.model = model
        fam.loss_fn = loss_fn
        fam.optimizer = optimizer

## features including assessment -> pathology (calc)

In [26]:
# default settings
fpc = Model()

fpc.epochs = epochs
fpc.batch_size = batch_size
fpc.train_dataset, fpc.test_dataset, fpc.train_dataloader, fpc.test_dataloader = get_dataloaders(train_calc, test_calc, "p", batch_size)

fpc.input_size = fpc.train_dataloader.dataset.X.shape[1]
fpc.hidden_size = hidden_size
fpc.num_classes = 3

fpc.lr = 1e-3

In [52]:
# default model
fpc.model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
fpc.loss_fn = nn.CrossEntropyLoss()
fpc.optimizer = torch.optim.SGD(fpc.model.parameters(), lr=fpc.lr)

for t in range(fpc.epochs):
    train_loop(fpc.train_dataloader, fpc.model, fpc.loss_fn, fpc.optimizer)
    accuracy = test_loop(fpc.test_dataloader, fpc.model, fpc.loss_fn)[0]
fpc.accuracy = accuracy
fpc.accuracy

63.8

In [53]:
# Mini-batch gradient descent
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fpc.lr, momentum=0.9)

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

New best model found with accuracy:  65.0
New best model found with accuracy:  65.3
New best model found with accuracy:  66.0
New best model found with accuracy:  66.9


In [54]:
# SGD momentum
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fpc.lr, momentum=0.9)

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

New best model found with accuracy:  67.2
New best model found with accuracy:  67.5
New best model found with accuracy:  68.1


In [55]:
# SGD Nestrov
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fpc.lr, momentum=0.9, nesterov=True)

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

In [56]:
# Adam
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

New best model found with accuracy:  68.4
New best model found with accuracy:  68.7
New best model found with accuracy:  69.0


In [57]:
# Adagrad
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adagrad(model.parameters())

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

New best model found with accuracy:  69.3


In [58]:
# Adadelta
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adadelta(model.parameters())

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

In [59]:
# Adamax
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adamax(model.parameters())

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

In [60]:
# RMSprop
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

New best model found with accuracy:  69.6
New best model found with accuracy:  71.2


RMSprop

In [65]:
# learning rate scheduler: StepLR
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

New best model found with accuracy:  72.7


In [66]:
# learning rate scheduler: ReduceLROnPlateau
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0)

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer
    scheduler.step(accuracy)

learning rate scheduler: StepLR

In [67]:
# Number of Neurons: int((input_size + num_classes)/2)
n = int((fpc.input_size + fpc.num_classes)/2)
for i in range(40):
    model = FNN(fpc.input_size, n, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

In [68]:
# Number of Neurons: int(2/3*input_size + num_classes)
n = int(2/3*fpc.input_size + fpc.num_classes)
for i in range(40):
    model = FNN(fpc.input_size, n, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

1

In [69]:
# Number of Layers: 2
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, hidden_layers=2).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0)

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer
    scheduler.step(accuracy)

In [71]:
fpc.hidden_layers = 1

1

In [72]:
# Batch size: 32 (default is 64)
train_dataset, test_dataset, train_dataloader, test_dataloader = get_dataloaders(train_calc, test_calc, "p", 32)

for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.train_dataset, fpc.test_dataset, fpc.train_dataloader, fpc.test_dataloader = fpc.train_dataset, test_dataset, train_dataloader, test_dataloader
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer
        fpc.batch_size = 32

c:\Users\sonog\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


64

In [73]:
# number of epochs (default): 10 -> increase until 3-
e = fpc.epochs
for i in range(3):
    for j in range(10):
        model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes).to(device)
        optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
        scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

        for t in range(e):
            scheduler.step()
            train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
            accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
        
        if accuracy > fpc.accuracy:
            print("New best epochs: ", e)
            print("New best model found with accuracy: ", accuracy)
            fpc.accuracy = accuracy
            fac.epochs = e
            fpc.model = model
            fpc.loss_fn = loss_fn
            fpc.optimizer = optimizer
    e += 1

c:\Users\sonog\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


10

In [74]:
# Activation Function: leaky_relu (default is ReLU)
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, activation="leaky_relu").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

In [75]:
# Activation Function: tanh (default is ReLU)
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, activation="tanh").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

In [76]:
# Activation Function: sigmoid (default is ReLU)
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, activation="sigmoid").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

ReLu

In [78]:
# weight_init : zero (default is lecun)
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, weight_init="zero").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

In [79]:
# weight_init : normal (default is lecun) # no lr scheduler and 2 hidden layers by mistake
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, weight_init="normal", hidden_layers=2).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

New best model found with accuracy:  73.3
New best model found with accuracy:  75.5


In [82]:
# weight_init : normal (default is lecun)
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, weight_init="normal").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

In [83]:
# weight_init : xavier (default is lecun)
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, weight_init="xavier").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

In [84]:
# weight_init : kaiming (default is lecun)
for i in range(40):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, weight_init="kaiming").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.1)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    for t in range(fpc.epochs):
        scheduler.step()
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

Normal without lr scheduler and hidden_layers=2

In [86]:
# L2 Regularization
# Define the hyperparameter search space
epochs_candidates = [10, 20, 30]
l2_lambda_candidates = [0.001, 0.01, 0.1]

for num_epochs, l2_lambda in itertools.product(epochs_candidates, l2_lambda_candidates):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, hidden_layers=2, weight_init="normal").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

    for t in range(num_epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        print("New best hyperparameters: ", num_epochs, l2_lambda)
        
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer
        
        fac.epochs = num_epochs
        fac.l2_lambda = l2_lambda

In [91]:
# L1 Regularization
epochs_candidates = [10, 20, 30]
l1_lambda_candidates = [0.001, 0.01, 0.1]

for num_epochs, l1_lambda in itertools.product(epochs_candidates, l1_lambda_candidates):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, hidden_layers=2, weight_init="normal").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=l1_lambda)

    for t in range(num_epochs):
        train_L1(fpc.train_dataloader, model, loss_fn, optimizer, l1_lambda)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        print("New best hyperparameters: ", num_epochs, l2_lambda)
        
        fpc.accuracy = accuracy
        
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer
        
        fpc.epochs = num_epochs
        fpc.l1_lambda = l1_lambda

No regularization

In [92]:
for i in range(100):
    model = FNN(fpc.input_size, fpc.hidden_size, fpc.num_classes, weight_init="normal", hidden_layers=2).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpc.epochs):
        train_loop(fpc.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpc.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpc.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpc.accuracy = accuracy
        fpc.model = model
        fpc.loss_fn = loss_fn
        fpc.optimizer = optimizer

## features including assessment -> pathology (mass)

In [52]:
# default settings
fpm = Model()

fpm.epochs = epochs
fpm.batch_size = batch_size
fpm.train_dataset, fpm.test_dataset, fpm.train_dataloader, fpm.test_dataloader = get_dataloaders(train_mass, test_mass, "p", batch_size)

fpm.input_size = fpm.train_dataloader.dataset.X.shape[1]
fpm.hidden_size = hidden_size
fpm.num_classes = 3

fpm.lr = 1e-3

In [53]:
# default model
fpm.model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
fpm.loss_fn = nn.CrossEntropyLoss()
fpm.optimizer = torch.optim.SGD(fpm.model.parameters(), lr=fpm.lr)

for t in range(fpm.epochs):
    train_loop(fpm.train_dataloader, fpm.model, fpm.loss_fn, fpm.optimizer)
    accuracy = test_loop(fpm.test_dataloader, fpm.model, fpm.loss_fn)[0]
fpm.accuracy = accuracy
fpm.accuracy

63.5

In [54]:
# Mini-batch gradient descent
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fpm.lr)

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

New best model found with accuracy:  64.0


In [55]:
# SGD momentum
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fpm.lr, momentum=0.9)

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

New best model found with accuracy:  69.6
New best model found with accuracy:  70.6


In [56]:
# SGD Nestrov
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=fpm.lr, momentum=0.9, nesterov=True)

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [57]:
# Adam
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [58]:
# Adagrad
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adagrad(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [59]:
# Adadelta
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adadelta(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [60]:
# Adamax
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adamax(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [61]:
# RMSprop
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

New best model found with accuracy:  70.9
New best model found with accuracy:  72.5


RMSprop

In [69]:
# Number of Neurons: int(2/3*input_size + num_classes)
n = int(2/3*fpm.input_size + fpm.num_classes)

for i in range(40):
    model = FNN(fpm.input_size, n, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [70]:
# Number of Neurons: int(2/3*input_size + num_classes)
n = int(2/3*fpm.input_size + fpm.num_classes)

for i in range(40):
    model = FNN(fpm.input_size, n, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [64]:
# Number of Neurons: default

for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

New best model found with accuracy:  73.3


512

In [65]:
# Number of Layers: 2
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, hidden_layers=2).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

1

In [67]:
# Batch size: 32 (default is 64)
train_dataset, test_dataset, train_dataloader, test_dataloader = get_dataloaders(train_mass, test_mass, "p", 32)

for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.train_dataset, fpm.test_dataset, fpm.train_dataloader, fpm.test_dataloader = train_dataset, test_dataset, train_dataloader, test_dataloader
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

64

In [32]:
# number of epochs (default): 10 -> increase until 3-
e = fpm.epochs
for i in range(3):
    for j in range(10):
        model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.RMSprop(model.parameters())

        for t in range(e):
            train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
            accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
        
        if accuracy > fpm.accuracy:
            print("New best epochs: ", e)
            print("New best model found with accuracy: ", accuracy)
            fpm.accuracy = accuracy
            fpm.epochs = e
            fpm.model = model
            fpm.loss_fn = loss_fn
            fpm.optimizer = optimizer
    e += 1

10

In [68]:
# Activation Function: leaky_relu (default is ReLU)
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, activation="leaky_relu").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [71]:
# Activation Function: tanh (default is ReLU)
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, activation="tanh").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [72]:
# Activation Function: sigmoid (default is ReLU)
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, activation="sigmoid").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

ReLu

In [73]:
# weight_init : zero (default is lecun)
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, weight_init="zero").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [74]:
# weight_init : normal (default is lecun)
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, weight_init="normal").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [75]:
# weight_init : xavier (default is lecun)
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, weight_init="xavier").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

New best model found with accuracy:  73.8


In [76]:
# weight_init : kaiming (default is lecun)
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, weight_init="kaiming").to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

In [77]:
# weight_init : normal (default is lecun) # same condition as best of fpc
for i in range(40):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, weight_init="normal", hidden_layers=2).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters())

    for t in range(fpm.epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer

Lecun

In [78]:
# L2 Regularization
# Define the hyperparameter search space
epochs_candidates = [10, 20, 30]
l2_lambda_candidates = [0.001, 0.01, 0.1]

for num_epochs, l2_lambda in itertools.product(epochs_candidates, l2_lambda_candidates):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, weight_init="normal", hidden_layers=2).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

    for t in range(num_epochs):
        train_loop(fpm.train_dataloader, model, loss_fn, optimizer)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        print("New best hyperparameters: ", num_epochs, l2_lambda)
        
        fpm.accuracy = accuracy
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer
        
        fpm.epochs = num_epochs
        fpm.l2_lambda = l2_lambda

In [79]:
# L1 Regularization
epochs_candidates = [10, 20, 30]
l1_lambda_candidates = [0.001, 0.01, 0.1]

for num_epochs, l1_lambda in itertools.product(epochs_candidates, l1_lambda_candidates):
    model = FNN(fpm.input_size, fpm.hidden_size, fpm.num_classes, weight_init="normal", hidden_layers=2).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=l1_lambda)

    for t in range(num_epochs):
        train_L1(fpm.train_dataloader, model, loss_fn, optimizer, l1_lambda)
        accuracy = test_loop(fpm.test_dataloader, model, loss_fn)[0]
    
    if accuracy > fpm.accuracy:
        print("New best model found with accuracy: ", accuracy)
        print("New best hyperparameters: ", num_epochs, l1_lambda)
        
        fpm.accuracy = accuracy
        
        fpm.model = model
        fpm.loss_fn = loss_fn
        fpm.optimizer = optimizer
        
        fpm.epochs = num_epochs
        fpm.l1_lambda = l1_lambda

No regularization

# Accuracies

In [489]:
# fac.accuracy

77.3

In [490]:
# fam.accuracy

64.0

In [94]:
# fpc.accuracy

75.5

In [80]:
fpm.accuracy

73.8

# Save the models

In [434]:
# torch.save(fac.model.state_dict(), "FNN_fac.pt")

In [435]:
# torch.save(fam.model.state_dict(), "FNN_fam.pt")

In [95]:
# torch.save(fpc.model.state_dict(), "FNN_fpc.pt")

In [81]:
torch.save(fpm.model.state_dict(), "FNN_fpm.pt")